# FIRST

We now demonstrate how to use `FIRST` for factor importance ranking and selection. If you have not installed `pyfirst`, please uncomment and run `%pip install pyfirst` below before proceeding. 

In [1]:
# %pip install pyfirst

## Imports

In [2]:
import numpy as np
from pyfirst import FIRST
from sklearn.datasets import make_friedman1

## Simulate Data

We simulate noisy data from the Friedman function 

$$
    y = f(X) + \epsilon = 10\sin(\pi X_{1}X_{2}) + 20(X_{3}-0.5)^2 + 10X_{4} + 5X_{5} + \epsilon,
$$

where the input $X$ are independent features uniformly distributed on unit hypercube and $\epsilon\sim\mathcal{N}(0,1)$ is independent of input $X$. Here only the first 5 features are used, and the remaining are independent of $y$. 

In [3]:
X, y = make_friedman1(n_samples=10000, n_features=10, noise=1.0, random_state=43)

## Run FIRST

In [4]:
FIRST(X, y)

array([0.2648618 , 0.2387856 , 0.07689935, 0.32580823, 0.0700073 ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

As expected, `FIRST` identifies the first 5 factors as important, and the remaining are not taking part in the prediction of the response.

## Speeding Up FIRST

### Parallel Computation

If multiple processors are available, `FIRST` is supported to run in parallel for acceleration via the argument `n_jobs`.

In [5]:
FIRST(X, y, n_jobs=4)

array([0.2648618 , 0.2387856 , 0.07689935, 0.32580823, 0.0700073 ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

### Approximate Nearest-Neighbor Search

`FIRST` requires many nearest-neighbor searches. Faiss (Douze et al., 2024) is used for efficient nearest-neighbor search, with approximate search (`approx_knn=True`) by the inverted file index (IVF) is also supported in the implementation. IVF reduces the search scope through first clustering data into Voronoi cells. 

In [6]:
FIRST(X, y, approx_knn=True)

array([0.2648618 , 0.23877103, 0.07689935, 0.32580823, 0.0700073 ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

### Using Subsamples

The use of subsamples to accelerate computation of the outer loop expectation is available via the argument `n_mc`. Both random and twinning (Vakayil and Joseph, 2022) subsamples are available, where twinning subsamples could provide better approximation for the full data at a higher computational cost. 

#### Random Subsamples

In [7]:
np.random.seed(43)
FIRST(X, y, n_mc=1000)

array([0.26479634, 0.23567279, 0.07756636, 0.33297155, 0.064703  ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

#### Twinning Subsamples

In [8]:
np.random.seed(43)
FIRST(X, y, n_mc=1000, twin_mc=True)

array([0.28075163, 0.2553532 , 0.06957491, 0.3351466 , 0.07408396,
       0.        , 0.        , 0.        , 0.        , 0.        ])

### All the Tricks Together

Using all the speed-up tricks, we can easily run `FIRST` on dataset with a ***million*** instances.

In [9]:
X, y = make_friedman1(n_samples=1000000, n_features=10, noise=1.0, random_state=43)
FIRST(X, y, n_mc=1000, approx_knn=True, n_jobs=4)

array([0.26729722, 0.23971214, 0.08612567, 0.33789522, 0.08616549,
       0.        , 0.        , 0.        , 0.        , 0.        ])

For more details about `FIRST`, please Huang and Joseph (2024).

## References

Huang, C., & Joseph, V. R. (2024). Factor Importance Ranking and Selection using Total Indices. arXiv preprint arXiv:2401.00800.

Douze, M., Guzhva, A., Deng, C., Johnson, J., Szilvasy, G., Mazaré, P.E., Lomeli, M., Hosseini, L., & Jégou, H., (2024). The Faiss library. arXiv preprint arXiv:2401.08281.
    
Vakayil, A., & Joseph, V. R. (2022). Data twinning. Statistical Analysis and Data Mining: The ASA Data Science Journal, 15(5), 598-610.